In [ ]:
import numpy as np
import pandas as pd

# **FCCN**

### **Activation and Initialisation**

In [ ]:
def ReLU(Z):
    return np.maximum(0, Z)

def params(dimn):

    W = [None]
    b = [None]

    for i in range(1, len(dimn)):
        he = np.sqrt(2.0 / dimn[i-1])

        W.append(np.random.randn(dimn[i], dimn[i-1]) * he)
        b.append(np.zeros((dimn[i], 1)))

    return W, b

### **Forward Pass**

In [ ]:
def forward(X, W, b):
    A = [X]
    Z = [None]

    for i in range(1, len(W)):
        Zi = np.dot(W[i], A[i-1]) + b[i]
        Z.append(Zi)

        if i == len(W) - 1:
            Ai = Zi
        else:
            Ai = ReLU(Zi)

        A.append(Ai)
    return A, Z

### **Backward Propagation**

In [ ]:
def backprop(Y, W, b, A, Z, lr):
    n = Y.shape[1]
    dZ = A[-1] - Y 

    for i in range(len(W) - 1, 0, -1):
        dW = np.dot(dZ, A[i-1].T) / n
        db = np.sum(dZ, axis=1, keepdims=True) / n

        if i > 1:
            dA_prev = np.dot(W[i].T, dZ)
            dZ = dA_prev * (Z[i-1] > 0)

        W[i] -= lr * dW
        b[i] -= lr * db
        
    return W, b

### **Training**

In [ ]:
def train_model(X_train, y_train, X_test, y_test, lr=0.05, epochs=1000):

    ip_dimn = X_train.shape[0]
    layer_dimn = [ip_dimn, 64, 32, 1] 
    W, b = params(layer_dimn)
    
    for i in range(epochs):
        A, Z = forward(X_train, W, b)
        W, b = backprop(y_train, W, b, A, Z, lr)
        
        if i % 100 == 0:
            loss = np.mean(np.square(A[-1] - y_train))
            print(f"Iter {i}: Loss {loss:.4f}")

    A_test, _ = forward(X_test, W, b)
    test_acc = np.mean((A_test[-1] > 0.5).astype(int) == y_test) * 100
    print(f"Test Accuracy: {test_acc:.2f}%")

# **Implementation**

### **Loading**

In [ ]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 
           'education-num', 'marital-status', 'occupation', 'relationship', 
           'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'income']

train_df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?', skipinitialspace=True)
test_df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skipinitialspace=True, skiprows=1)

train_df['income'] = train_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)

test_df['income'] = test_df['income'].str.replace('.', '', regex=False)
test_df['income'] = test_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)

train_df = train_df.drop(['fnlwgt', 'education'], axis=1)
test_df = test_df.drop(['fnlwgt', 'education'], axis=1)

train_df = train_df.dropna()
test_df = test_df.dropna()

categ = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
train = pd.get_dummies(train_df, columns=categ, drop_first=True)
test = pd.get_dummies(test_df, columns=categ, drop_first=True)

test = test.reindex(columns=train.columns, fill_value=0)

feats = [col for col in train.columns if col != 'income']

### **Without Scaling**

In [ ]:
print("--- Raw Data ---")

X_train_raw = train[feats].values.astype(float)
y_train_raw = train['income'].values.reshape(-1, 1)

X_test_raw = test[feats].values.astype(float)
y_test_raw = test['income'].values.reshape(-1, 1)

X_train_raw, y_train_raw = X_train_raw.T, y_train_raw.T
X_test_raw, y_test_raw = X_test_raw.T, y_test_raw.T

train_model(X_train_raw, y_train_raw, X_test_raw, y_test_raw, lr=0.01)

### **With Min-Max Scaling**

In [ ]:
print("--- Min-Max Scaling ---")

X_train_mm = train[feats].values.astype(float)
y_train_mm = train['income'].values.reshape(-1, 1)

X_test_mm = test[feats].values.astype(float)
y_test_mm = test['income'].values.reshape(-1, 1)

mini = X_train_mm.min(axis=0)
maxi = X_train_mm.max(axis=0)

X_train_mm = (X_train_mm - mini) / (maxi - mini)
X_test_mm = (X_test_mm - mini) / (maxi - mini)

X_train_mm, y_train_mm = X_train_mm.T, y_train_mm.T
X_test_mm, y_test_mm = X_test_mm.T, y_test_mm.T

train_model(X_train_mm, y_train_mm, X_test_mm, y_test_mm, lr=0.1)